The goal of this exercise is to look at the effect of tax changes on different segments of the population and the aggregate effect. Please see the [video](https://www.loom.com/share/1b0cc6479ab54277b879f849a836ea20) for more details.

In [ ]:
# Copy of tax-calculator functions
std_pa = 12509

def calculate_bracketed_tax (amount, brackets, rates):
  tax_total=0
  for bracket_l, bracket_u, rate in zip(brackets, brackets[1:], rates):
    tax_total += rate*max(0,min(bracket_u-bracket_l,amount-bracket_l))
  tax_total += max(0,amount-brackets[len(brackets)-1])*rates[len(rates)-1]
  return tax_total

def calculate_pa(gross, init_pa = std_pa):
  return init_pa - 0.5*max(0,min(2*init_pa,gross-100000))

def income_tax(gross, init_pa=std_pa):
  pa = calculate_pa(gross, init_pa)
  return calculate_bracketed_tax(gross,[pa,pa+37500,pa+150000],[0.2,0.4,0.45])

def national_insurance(gross):
  return calculate_bracketed_tax(gross,[9500,50000],[0.12,0.02])

def pay_after_tax(gross, init_pa=std_pa):
  return gross-income_tax(gross, init_pa)-national_insurance(gross)

In [ ]:
import pandas as pd
full_data = pd.read_csv("income_by_percentile.csv", index_col='percentile', thousands=',')
gross_incomes_1718=full_data['2017-18']
gross_incomes_1718

percentile
1      11700
2      12000
3      12300
4      12500
5      12800
       ...  
95     76800
96     86000
97     98600
98    121000
99    175000
Name: 2017-18, Length: 99, dtype: int64

In [ ]:
lowest = gross_incomes_1718.loc[1]
lowest_after_tax = pay_after_tax(lowest)
lowest, lowest_after_tax

(11700, 11436.0)

In [ ]:
after_tax_1718=gross_incomes_1718.apply(pay_after_tax)
tax_table=pd.concat([gross_incomes_1718.rename('before tax'), after_tax_1718.rename('after tax')], axis=1)
tax_table.loc[50]

before tax    24400.0
after tax     20233.8
Name: 50, dtype: float64

In [ ]:
tax_paid = gross_incomes_1718-after_tax_1718
tax_paid.sum()*32000000/100

219711040000.00003

In [ ]:
def new_tax(gross):
  return